Skboost

Dit is het gemakkelijkste 

In [1]:
!pip uninstall -y skboost
!pip install git+https://github.com/tibovanheule/skboost

Found existing installation: skboost 0.2.1
Uninstalling skboost-0.2.1:
  Successfully uninstalled skboost-0.2.1
  Cloning https://github.com/tibovanheule/skboost to /tmp/pip-req-build-zdpb89y5
  Running command git clone --filter=blob:none --quiet https://github.com/tibovanheule/skboost /tmp/pip-req-build-zdpb89y5
  Resolved https://github.com/tibovanheule/skboost to commit 14507560afbfc50cb5a6417cab8e4023b3b58941
  Preparing metadata (setup.py) ... done
  Created wheel for skboost: filename=skboost-0.2.1-cp38-cp38-linux_x86_64.whl size=1470579 sha256=9c9384d2289ad9475db49aa433c280e45c70c893773e0dc9ed53c0ed716258b8
  Stored in directory: /tmp/pip-ephem-wheel-cache-4jjn6je3/wheels/36/07/75/fd1bf14124b634197919ed0b16c8a27d35126499066335017e
Successfully built skboost


In [2]:
import skboost
#help(skboost.milboost.softmax)

In [3]:
from skboost import milboost
help(milboost.classifier.MILBoostClassifier)

Help on class MILBoostClassifier in module skboost.milboost.classifier:

class MILBoostClassifier(sklearn.base.ClassifierMixin, sklearn.ensemble._weight_boosting.BaseWeightBoosting)
 |  MILBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=10), softmax=None, n_estimators=50, learning_rate=1.0, random_state=None, verbose=False)
 |  
 |  Mixin class for all classifiers in scikit-learn.
 |  
 |  Method resolution order:
 |      MILBoostClassifier
 |      sklearn.base.ClassifierMixin
 |      sklearn.ensemble._weight_boosting.BaseWeightBoosting
 |      sklearn.ensemble._base.BaseEnsemble
 |      sklearn.base.MetaEstimatorMixin
 |      sklearn.base.BaseEstimator
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, base_estimator=DecisionTreeClassifier(max_depth=10), softmax=None, n_estimators=50, learning_rate=1.0, random_state=None, verbose=False)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  __str__(self)
 |      Retu

In [4]:
from loader import parse_c45, bag_set
from __future__ import print_function, division
from sklearn.model_selection import StratifiedKFold
from skboost.milboost.classifier import MILBoostClassifier
from skboost.milboost.softmax import *
from score import result
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import time


In [5]:
# Load list of C4.5 Examples
for dataset in ['fox','mutagenesis-atoms','mutagenesis-bonds','mutagenesis-chains','eastWest','elephant','tiger','westEast','musk1']:
    print(dataset)
    example_set = parse_c45(dataset)

    # Get stats to normalize data
    raw_data = np.array(example_set.to_float())
    data_mean = np.average(raw_data, axis=0)
    data_std  = np.std(raw_data, axis=0)
    data_std[np.nonzero(data_std == 0.0)] = 1.0
    def normalizer(ex):
        ex = np.array(ex)
        if dataset == 'musk':
            return ex[2:]
        return ex[1:]


    # Group examples into bags
    bagset = bag_set(example_set)

    molecule_names = []
    conformation_names = []
    bags = []
    labels = []

    bag_id = 0
    for bag in bagset: 
        for ro in bag.to_float(normalizer):
            if ro[0] not in molecule_names:
                molecule_names.append(ro[0])
                bag_id += 1
            labels.append(int(((float(ro[-1]) * 2) - 1)))
            conformation_names.append(ro[1])
            bags.append(list(map(float, ro[2:-1])))

    bags = np.array(bags,dtype="float")
    labels = np.array(labels,dtype="int")

    fold = StratifiedKFold(n_splits=5, shuffle=False, random_state=None)
    splittt = 1  
    for train_index, test_index in fold.split(bags,labels):
        X_train, X_test = bags[train_index], bags[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        epoch_time = time.time()
        model = MILBoostClassifier(softmax=LogSumExponential(5.0))
        model.fit(X_train,y_train)
        start = time.time()
        predicted = model.predict(X_test)
        time_ep = time.time() - start
        result("MILboost LogSumExponential",y_test,predicted,time_ep, splittt,dataset)
        splittt += 1

    for train_index, test_index in fold.split(bags,labels):
        X_train, X_test = bags[train_index], bags[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        epoch_time = time.time()
        model = MILBoostClassifier(softmax=GeneralizedMean(5.0))
        model.fit(X_train,y_train)
        start = time.time()
        predicted = model.predict(X_test)
        time_ep = time.time() - start
        result("MILboost",y_test,predicted,time_ep, splittt,dataset)
        splittt += 1

#bag_id = 0
for bag in bagset: 
    for ro in bag.to_float(normalizer):
        if ro[0] not in molecule_names:
            molecule_names.append(ro[0])
            #bag_id += 1
        labels.append(int(((float(ro[-1]) * 2) - 1)))
        conformation_names.append(ro[1])
        bags.append(list(map(float, ro[2:-1])))

    for train_index, test_index in fold.split(bags,labels):
        X_train, X_test = bags[train_index], bags[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        epoch_time = time.time()
        model = MILBoostClassifier(softmax=ISR())
        model.fit(X_train,y_train)
        start = time.time()
        predicted = model.predict(X_test)
        time_ep = time.time() - start
        result("MILboost ISR",y_test,predicted,time_ep, splittt,dataset)
        splittt += 1
    

fox
mutagenesis-atoms
mutagenesis-bonds


/home/lotte/Documents/2MA/ML/MachineLearning/score.py:17: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots(figsize=(7.5, 7.5))


mutagenesis-chains
eastWest


/home/lotte/Documents/2MA/ML/MachineLearning/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/lotte/Documents/2MA/ML/MachineLearning/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/lotte/Documents/2MA/ML/MachineLearning/venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _wa

elephant
tiger
westEast


...

<Figure size 750x750 with 0 Axes>

<Figure size 750x750 with 0 Axes>

<Figure size 750x750 with 0 Axes>

<Figure size 750x750 with 0 Axes>

<Figure size 750x750 with 0 Axes>

In [ ]:
import matplotlib.pyplot as plt

x = np.linspace(0.0, 1.0, num=100)
t = np.vstack([x, 1 - x]).T
true_t = np.max(t, axis=1)

lse_5 = list(map(LogSumExponential(5.0).f, t))
lse_20 = list(map(LogSumExponential(20.0).f, t))
nor = list(map(NoisyOR().f, t))
isr = list(map(ISR().f, t))
gm_5 = list(map(GeneralizedMean(5.0).f, t))
gm_20 = list(map(GeneralizedMean(20.0).f, t))

plt.plot(x, true_t, 'k--', label='True')
plt.plot(x, lse_5, 'b', label='LSE(5)')
plt.plot(x, lse_20, 'y', label='LSE(20)')
plt.plot(x, nor, 'r', label='NOR')
plt.plot(x, isr, 'g', label='ISR')
plt.plot(x, gm_5, 'm', label='GM(5)')
plt.plot(x, gm_20, 'c', label='GM(20)')

plt.legend()
plt.xlabel('v')
plt.ylabel('g_{l}(v_{l})')

plt.show()
plt.savefig("softmax",dpi=300)
